In [7]:
import numpy as np
import pandas as pd

from collections import defaultdict
from tqdm import tqdm

import random

import gym
import adaptive_tutor

In [8]:
import numpy as np
action_space = np.load('adaptive_tutor/action_space.npy', allow_pickle=True)

In [3]:
class TutorStatefulAgent:
    def __init__(
        self,
        learning_rate: float,
        initial_epsilon: float,
        epsilon_decay: float,
        final_epsilon: float,
        discount_factor: float = 0.95,
    ):
        """Initialize a Reinforcement Learning agent with an empty dictionary
        of state-action values (q_values), a learning rate and an epsilon.

        Args:
            learning_rate: The learning rate
            initial_epsilon: The initial epsilon value
            epsilon_decay: The decay for epsilon
            final_epsilon: The final epsilon value
            discount_factor: The discount factor for computing the Q-value
        """

        self.action_space = list(np.load('adaptive_tutor/action_space.npy', allow_pickle=True))
        self.q_values = defaultdict(lambda: np.zeros(len(self.action_space)))

        self.lr = learning_rate
        self.discount_factor = discount_factor

        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon

        self.training_error = []

    def _create_rating_bracket(self, row):
        if row<1300:
            return 'lt_1300'
        elif 1300<=row<1700:
            return '1300-1800'
        else:
            return 'gt_1700'
    
    def _get_agent_state(self, obs):
        return tuple([self._create_rating_bracket(val) for val in obs['themes_covered']])

    def get_action(self, obs: tuple[int, int, bool]) -> int:
        """
        Returns the best action with probability (1 - epsilon)
        otherwise a random action with probability epsilon to ensure exploration.
        """
        # with probability epsilon return a random action to explore the environment
        obs = self._get_agent_state(obs)
        if np.random.random() < self.epsilon:
            print("Exploring")
            return random.choice(range(len(self.action_space)))

        # with probability (1 - epsilon) act greedily (exploit)
        else:
            return (np.argmax(self.q_values[obs]))

    def update(
        self,
        obs: tuple[int, int, bool],
        action: int,
        reward: float,
        terminated: bool,
        next_obs: tuple[int, int, bool],
    ):
        """Updates the Q-value of an action."""
        obs = self._get_agent_state(obs)
        next_obs = self._get_agent_state(next_obs)
        future_q_value = (not terminated) * np.max(self.q_values[next_obs])
        temporal_difference = (
            reward + self.discount_factor * future_q_value - self.q_values[obs][action]
        )

        self.q_values[obs][action] = (
            self.q_values[obs][action] + self.lr * temporal_difference
        )
        self.training_error.append(temporal_difference)

    def decay_epsilon(self):
        self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)

In [4]:
class TutorAgent:
    def __init__(
        self,
        learning_rate: float,
        initial_epsilon: float,
        epsilon_decay: float,
        final_epsilon: float,
        discount_factor: float = 0.95,
    ):
        """Initialize a Reinforcement Learning agent with an empty dictionary
        of state-action values (q_values), a learning rate and an epsilon.

        Args:
            learning_rate: The learning rate
            initial_epsilon: The initial epsilon value
            epsilon_decay: The decay for epsilon
            final_epsilon: The final epsilon value
            discount_factor: The discount factor for computing the Q-value
        """
        self.action_space = list(np.load('adaptive_tutor/action_space.npy', allow_pickle=True))
        self.action_space_dict = {action: i for i,action in enumerate(self.action_space)}
        self.q_values = np.zeros(len(self.action_space))

        self.lr = learning_rate
        self.discount_factor = discount_factor

        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon

        self.training_error = []

    def get_action(self) -> int:
        """
        Returns the best action with probability (1 - epsilon)
        otherwise a random action with probability epsilon to ensure exploration.
        """
        # with probability epsilon return a random action to explore the environment
        if np.random.random() < self.epsilon:
            print("Exploring")
            return random.choice(self.action_space)

        # with probability (1 - epsilon) act greedily (exploit)
        else:
            return self.action_space[np.argmax(self.q_values)]
    

    def update(
        self,
        obs: tuple[int, int, bool],
        action: int,
        reward: float,
        terminated: bool,
        next_obs: tuple[int, int, bool],
    ):
        """Updates the Q-value of an action."""
        
        future_q_value = (not terminated) * np.max(self.q_values)
        temporal_difference = (
            reward + self.discount_factor * future_q_value - self.q_values[self.action_space.index(action)]
        )

        self.q_values[self.action_space.index(action)] = (
            self.q_values[self.action_space.index(action)] + self.lr * temporal_difference
        )
        self.training_error.append(temporal_difference)

    def decay_epsilon(self):
        self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)

In [5]:
# Hyperparameters
learning_rate = 0.01
n_episodes = 1000
start_epsilon = 0.5
epsilon_decay = start_epsilon / (n_episodes / 2) # Reduce the exploration over time
final_epsilon = 0.1

agent = TutorStatefulAgent(
    learning_rate=learning_rate,
    initial_epsilon=start_epsilon,
    epsilon_decay=epsilon_decay,
    final_epsilon=final_epsilon,
)

In [6]:
# Load custom environment we created 
env = gym.make('adaptive_tutor/PuzzleTutorEnv-v0', render_mode=None) 

# Set to initial state
env.reset()

for episode in tqdm(range(n_episodes)):
    obs, info = env.reset()
    done = False
    terminated = False
    # Play one episode
    step_counter = 0
    while not terminated:
        step_counter+=1
        action = agent.get_action(obs)
        next_obs, reward, terminated, truncated, info = env.step(action)
        print(action, reward)
        # Update the agent
        agent.update(obs, action, reward, terminated, next_obs)

        # Update the current observation, and whether the environment is done
        done = terminated or truncated
        obs = next_obs
    
    print("Number of steps: ", step_counter)

    agent.decay_epsilon()

/Users/shikharrastogi/opt/anaconda3/envs/rl/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
  0%|          | 0/1000 [00:00<?, ?it/s]<UciProtocol (pid=9603)>: stderr >>        _
<UciProtocol (pid=9603)>: stderr >> |   _ | |
<UciProtocol (pid=9603)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 22 2023
<UciProtocol (pid=9603)>: stderr >> Loading weights file from: /Users/shikharrastogi/AdaptiveChessTutorRL/maia_weights/maia_1100.pb
<UciProtocol (pid=9603)>: stderr >> Creating backend [metal]...
<UciProtocol (pid=9603)>: stderr >> Initialized metal backend on device Apple M1
/Users/shikharrastogi/opt/anaconda3/envs/rl/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool,

('1000-1100', 'advanced_tactical_themes') -2.473684210526316
Exploring
('gt_1900', 'checkmate_patterns') -2.0
('1000-1100', 'advanced_tactical_themes') -2.473684210526316
Exploring
('1800-1900', 'checkmate_patterns') -3.0
('1000-1100', 'checkmate_patterns') -3.0
Exploring
('1100-1200', 'king_safety_and_attack') -3.0
Exploring
('1200-1300', 'king_safety_and_attack') -2.3684210526315788
Exploring
('900-1000', 'tactical_themes') -2.526315789473684
Exploring
('900-1000', 'piece_specific_endgames') -3.0
('1000-1100', 'checkmating_tactics') -2.473684210526316
Exploring
('1500-1600', 'advanced_tactical_themes') -3.0
Exploring
('900-1000', 'advanced_tactical_themes') -2.526315789473684
('1000-1100', 'defensive_tactics') -2.473684210526316
('1000-1100', 'king_safety_and_attack') -2.473684210526316
Exploring
('900-1000', 'checkmate_patterns') -2.526315789473684
('1000-1100', 'advanced_tactical_themes') -2.473684210526316
Exploring
('1500-1600', 'piece_specific_endgames') -2.2105263157894735
Expl

  0%|          | 0/1000 [19:16<?, ?it/s]

('1600-1700', 'checkmate_patterns') -1.5410526315789475
Exploring


KeyboardInterrupt: 